## Cross in Tray 

In [1]:
using Pkg
Pkg.activate("../../.")
using Globtim
using DynamicPolynomials, DataFrames
using ProgressLogging

  Activating project at `~/Globtim.jl`


In [2]:
# Constants and Parameters
const n, a, b = 2, 10, 1
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm
f = CrossInTray # Objective function

CrossInTray (generic function with 1 method)

In [3]:
d = 10 # Initial Degree 
SMPL = 40 # Number of samples
center = [0.0, 0.0]
TR = test_input(n, center, (alpha, delta), tol_l2, (0.0, 0.0), scale_factor, 1.0, f)
pol_cheb = Constructor(TR, d, GN=SMPL, basis=:chebyshev)
pol_lege = Constructor(TR, d, GN=SMPL, basis=:legendre);

current L2-norm: 0.08153403450667797
Number of samples: 40
current L2-norm: 0.08999454706062496
Number of samples: 40


In [4]:
@polyvar(x[1:n]) # Define polynomial ring 

(Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}[x₁, x₂],)

In [ ]:
real_pts_cheb = solve_polynomial_system(x, n, d, pol_cheb.coeffs; basis=:chebyshev, bigint=true)
real_pts_lege = solve_polynomial_system(x, n, d, pol_lege.coeffs; basis=:legendre, bigint=true)
# Usage example:
df_cheb = process_critical_points(real_pts_cheb, f, scale_factor)
df_lege = process_critical_points(real_pts_lege, f, scale_factor)

Tracking 81 paths...   2%|▊                             |  ETA: 0:07:47



Tracking 81 paths... 100%|██████████████████████████████| Time: 0:00:12
  # paths tracked:                  81
  # non-singular solutions (real):  81 (81)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         81 (81)
Tracking 81 paths...   2%|▊                             |  ETA: 0:03:33



Tracking 81 paths...  12%|███▊                          |  ETA: 0:00:40



Tracking 81 paths...  22%|██████▋                       |  ETA: 0:00:20



Tracking 81 paths...  32%|█████████▋                    |  ETA: 0:00:13



Tracking 81 paths...  43%|█████████████                 |  ETA: 0:00:08



Tracking 81 paths...  54%|████████████████▎             |  ETA: 0:00:05



Tracking 81 paths...  62%|██████████████████▌           |  ETA: 0:00:04



Tracking 81 paths...  69%|████████████████████▊         |  ETA: 0:00:03



Tracking 81 paths...  78%|███████████████████████▍      |  ETA: 0:00:02



Tracking 

Row,x,y,z
,Float64,Float64,Float64
1,9.57872,9.57872,-9.83577
2,7.8509,9.5778,-12.3041
3,3.37724,9.59715,-11.5493
4,-3.37724,9.59715,-11.5493
5,-7.8509,9.5778,-12.3041
6,-9.57872,9.57872,-9.83577
7,-6.04338,9.5898,-11.1013
8,4.88966e-16,9.5931,-0.399639
9,6.04338,9.5898,-11.1013


In [ ]:
using GLMakie

# Extract coordinates and function values
# Change the coordinates to uniform grid
coords = pol_cheb.scale_factor * pol_lege.grid
z_coords = pol_lege.z

if size(coords)[2] == 2  # Plot if the dimensions are 2
    fig = Figure(size=(800, 600))
    ax = Axis3(fig[1, 1], title="Cross in Tray",
        xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

    # Scale the domain
    scale_factor = pol_cheb.scale_factor  # Using the scale factor from your polynomial

    # Define threshold for switching between scatter and surface
    point_threshold = 1000  # Adjust this value based on your needs
    sample_fraction = 0.2   # Fraction of points to use if exceeding threshold

    if length(z_coords) > point_threshold
        # Create a regular grid for surface plotting
        x_unique = sort(unique(coords[:, 1]))
        y_unique = sort(unique(coords[:, 2]))

        # Determine grid dimensions
        nx = length(x_unique)
        ny = length(y_unique)

        # Reshape data into a grid
        z_grid = reshape(z_coords, nx, ny)

        # Create surface plot with scaled domain
        surface!(ax, scale_factor .* x_unique, scale_factor .* y_unique, z_grid,
            colormap=:viridis,
            transparency=true,
            alpha=0.8)

        # Sample a fraction of points for scatter overlay
        n_points = length(z_coords)
        sample_indices = rand(1:n_points, Int(floor(n_points * sample_fraction)))

        # Plot sampled scatter points with scaled domain
        scatter!(ax, scale_factor .* coords[sample_indices, 1],
            scale_factor .* coords[sample_indices, 2],
            z_coords[sample_indices],
            markersize=2,
            color=:black,
            label="Sampled Data Points")
    else
        # Original scatter plot for smaller datasets with scaled domain
        scatter!(ax, scale_factor .* coords[:, 1],
            scale_factor .* coords[:, 2],
            z_coords,
            markersize=2,
            color=:black,
            label="Sampled Data Core")

        scatter!(ax, scale_factor .* coords[:, 1],
            scale_factor .* coords[:, 2],
            z_coords,
            markersize=4,
            color=z_coords,
            colormap=:viridis,
            label="Sampled Data Halo")
    end

    # Plot the critical points with scaled domain
    scatter!(ax, scale_factor .* df_cheb.x,
        scale_factor .* df_cheb.y,
        df_cheb.z,
        markersize=10,
        color=:orange,
        label="Chebyshev approximant critical points")

    scatter!(ax, scale_factor .* df_lege.x,
        scale_factor .* df_lege.y,
        df_lege.z,
        markersize=10,
        color=:yellow,
        label="Legendre approximant critical points")

    display(fig)
end

2024-12-07 12:49:00.604 julia[6649:69754] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-07 12:49:01.077 julia[6649:69754] +[IMKInputSession subclass]: chose IMKInputSession_Modern


GLMakie.Screen(...)